In [1]:
import pandas as pd

reviews_raw: pd.DataFrame = pd.read_csv("./data/letterboxd_250movie_reviews.csv")

reviews_raw.head()
valid_reviews = reviews_raw[reviews_raw['Rating'].notna()][reviews_raw['Status'] == 'Watched'][~reviews_raw['Review'].isna()].sample(frac=1, random_state=42)
valid_reviews.count()

/var/folders/6g/d16gbw5x5z9fk0rmvxw5z5q80000gn/T/ipykernel_75144/4114284106.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  valid_reviews = reviews_raw[reviews_raw['Rating'].notna()][reviews_raw['Status'] == 'Watched'][~reviews_raw['Review'].isna()].sample(frac=1, random_state=42)
/var/folders/6g/d16gbw5x5z9fk0rmvxw5z5q80000gn/T/ipykernel_75144/4114284106.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  valid_reviews = reviews_raw[reviews_raw['Rating'].notna()][reviews_raw['Status'] == 'Watched'][~reviews_raw['Review'].isna()].sample(frac=1, random_state=42)


Unnamed: 0    61096
Review        61096
Rating        61096
Date          61096
Status        61096
Movie         61096
dtype: int64

In [2]:
valid_reviews.head()

,Unnamed: 0,Review,Rating,Date,Status,Movie
43340,43340,A scathing political thriller from the great C...,★★★★★,19 Jul 2022,Watched,z
29334,29334,It's impossible to not respect how kids of a c...,★★★★½,04 Aug 2021,Watched,where-is-the-friends-house
72718,72718,"""There is only one golden rule: ""Do unto other...",★★★★★,21 Oct 2023,Watched,eureka
87904,87904,This is what I expect from screenplay when I s...,★★★★★,02 Jan 2022,Watched,happy-together-1997
57955,57955,"#madlad Mifune'If you would shed blood, then l...",★★★★,13 Apr 2021,Watched,throne-of-blood


remove non-english reviews:

In [3]:
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException

# Set seed for consistent results
DetectorFactory.seed = 0

def is_english(text: str) -> bool:
    try:
        return detect(text) == 'en'
    except LangDetectException:
        # Handle very short texts or detection failures
        return False

is_english(valid_reviews['Review'].iloc[0])
valid_reviews = valid_reviews[valid_reviews['Review'].apply(is_english)]

sample 300 reviews for each rating:

In [4]:
sample_for_each_rating = 300
sampled = []
for rating in valid_reviews['Rating'].unique():
    reviews_with_rating = valid_reviews[valid_reviews['Rating'] == rating]
    n = min(sample_for_each_rating, len(reviews_with_rating))
    print(f"Sampling {n} reviews with rating {rating}")
    sampled.append(reviews_with_rating.sample(n=n, random_state=42))

sampled_valid_reviews = pd.concat(sampled)
sampled_valid_reviews = sampled_valid_reviews.sample(frac=1, random_state=42) # shuffle again to avoid ordered by rating
sampled_valid_reviews = sampled_valid_reviews.drop(columns=['Status','Date']).rename(columns={'Unnamed: 0': 'original_id', 'Movie': 'movie'})
sampled_valid_reviews = sampled_valid_reviews.rename(columns={'Review': 'text', 'Rating': 'label'})
label2id = {
    "½": 0,
    "★": 1,
    "★½": 2,
    "★★": 3,
    "★★½": 4,
    "★★★": 5,
    "★★★½": 6,
    "★★★★": 7,
    "★★★★½": 8,
    "★★★★★": 9,
}
sampled_valid_reviews['label'] = sampled_valid_reviews['label'].apply(lambda x: label2id[x])
sampled_valid_reviews.head()

Sampling 300 reviews with rating ★★★★★
Sampling 300 reviews with rating ★★★★½
Sampling 300 reviews with rating ★★★★
Sampling 226 reviews with rating ★½
Sampling 300 reviews with rating ★★
Sampling 300 reviews with rating ★★★½
Sampling 300 reviews with rating ★★★
Sampling 216 reviews with rating ½
Sampling 300 reviews with rating ★★½
Sampling 251 reviews with rating ★


,original_id,text,label,movie
26543,26543,I finally went through the depths of cinephili...,1,satantango
73448,73448,"Of all the movies I haven’t seen, this is the ...",3,saving-private-ryan
35210,35210,Calling this 'bleak' or 'nihilistic' is more t...,1,an-elephant-sitting-still
9532,9532,I tend to look toward immersion and authentici...,9,le-trou
68308,68308,"Film #1 that was recommended to me,feel free t...",9,raging-bull


see some of the reviews:

In [5]:
def summarize(df_row)-> str:
    return f"Review: {df_row['text']} - Movie: {df_row['movie']}"

def check_rating(rating_id):
    return sampled_valid_reviews[sampled_valid_reviews['label'] == rating_id][sampled_valid_reviews['text'].str.len() < 250].apply(summarize, axis=1).head().tolist()

def log_sample_for_rating(rating):
    id = label2id[rating]
    samples = check_rating(id)
    print(f"Samples for rating {rating}:")
    for sample in samples:
        print(f' {sample}')
    print("\n")
log_sample_for_rating('★★★★★')
log_sample_for_rating('★★½')
log_sample_for_rating('½')

Samples for rating ★★★★★:
 Review: "It was bound to happen eventually." The natural elements overrun and invaded by bodies & blood red. War is unnatural, turning kings into fools & fools into kings. - Movie: ran
 Review: It’s a very personal piece and much about it is more mystifying than revelatory. Still, the leads are magnificent together. If nothing else, Aftersun will make you think about your relationship with your own father. - Movie: aftersun
 Review: So the climax of this film has to be the craziest ever for a court film. Its just so unpredictable and intense. I think its a really rewatchable court film and I certainly look forward to watching it again. - Movie: witness-for-the-prosecution-1957
 Review: First time on a big screen and I'm literally overwhelmed: after almost thirty years Satoshi Kon is still beyond the beyond. - Movie: perfect-blue
 Review: “This is my story. This is the sacrifice my father made. This was his gift to me.”Guido really is the best father - Movie: 

/var/folders/6g/d16gbw5x5z9fk0rmvxw5z5q80000gn/T/ipykernel_75144/1899875693.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return sampled_valid_reviews[sampled_valid_reviews['label'] == rating_id][sampled_valid_reviews['text'].str.len() < 250].apply(summarize, axis=1).head().tolist()
/var/folders/6g/d16gbw5x5z9fk0rmvxw5z5q80000gn/T/ipykernel_75144/1899875693.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return sampled_valid_reviews[sampled_valid_reviews['label'] == rating_id][sampled_valid_reviews['text'].str.len() < 250].apply(summarize, axis=1).head().tolist()
/var/folders/6g/d16gbw5x5z9fk0rmvxw5z5q80000gn/T/ipykernel_75144/1899875693.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return sampled_valid_reviews[sampled_valid_reviews['label'] == rating_id][sampled_valid_reviews['text'].str.len() < 250].apply(summarize, axis=1).head().tolist()


In [6]:
# save the sampled reviews to a new CSV file
sampled_valid_reviews.to_csv('./letterboxd_250movie_reviews_normalized_sampled.csv', index=False)

# divide into training, validation, and test sets
train_frac = 0.8
val_frac = 0.1
test_frac = 0.1
train_size = int(len(sampled_valid_reviews) * train_frac)
val_size = int(len(sampled_valid_reviews) * val_frac)   
test_size = int(len(sampled_valid_reviews) * test_frac)
train_set = sampled_valid_reviews[:train_size]
val_set = sampled_valid_reviews[train_size:train_size + val_size]
test_set = sampled_valid_reviews[train_size + val_size:]
print(f"Train set size: {len(train_set)}")
print(f"Validation set size: {len(val_set)}")
print(f"Test set size: {len(test_set)}")
train_set.to_csv('./data/letterboxd_250movie_reviews_train.csv', index=False)
val_set.to_csv('./data/letterboxd_250movie_reviews_val.csv', index=False)
test_set.to_csv('./data/letterboxd_250movie_reviews_test.csv', index=False)

Train set size: 2234
Validation set size: 279
Test set size: 280
